# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [15]:
import pandas as pd
import numpy as np
from scipy import stats
np.random.seed(42)

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

#### 1. What test is appropriate? Does the CLT apply?

A 2 sample $t$-test would be appropriate here, as we are comparing the mean callback responses for 2 independent samples. The Central Limit Theorum would apply due to the binary nature of the target variable, and can be represented as a Bernoulli distribution.

#### 2. Null and alternate hypothesis

$H_0$: Race does not affect the callback rate from employers

$H_A$: Race does affect the callback rate from employers

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

In [6]:
w.call.mean()

0.09650924056768417

In [7]:
b.call.mean()

0.0644763857126236

#### 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [11]:
# Your solution to Q3 here

def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)

    return bs_replicates

In [12]:
#confidence level: 95%
bootstrap_b = draw_bs_reps(b.call, np.mean, 10000)
bootstrap_w = draw_bs_reps(w.call, np.mean, 10000)

In [18]:
conf_int_b = np.percentile(bootstrap_b, [2.5, 97.5])
conf_int_w = np.percentile(bootstrap_w, [2.5, 97.5])
print('The confidence interval for black applicants is', conf_int_b[0], '-', conf_int_b[1])
print('The confidence interval for white applicants is', conf_int_w[0], '-', conf_int_w[1])
print('The margin of error for black applicants is ±', bootstrap_b.mean() - conf_int_b[0], '%')
print('The margin of error for white applicants is ±', bootstrap_w.mean() - conf_int_w[0], '%')

The confidence interval for black applicants is 0.05462012439966202 - 0.07433264702558517
The confidence interval for white applicants is 0.08501026779413223 - 0.10841888934373856
The margin of error for black applicants is ± 0.009796631141379478 %
The margin of error for white applicants is ± 0.0115847629904747 %


In [19]:
stats.ttest_ind(bootstrap_b, bootstrap_w)

Ttest_indResult(statistic=-410.99061396062075, pvalue=0.0)

Based on the 2 sample $t$-test for the bootstrap samples, the difference of the means is statistically significant. This means that we reject $H_0$ and assume that $H_A$ is true: <i>race affects the callback rate from employers.</i>

In [23]:
#frequentist approach
stats.ttest_ind(b.call, w.call)

Ttest_indResult(statistic=-4.114705290861751, pvalue=3.940802103128886e-05)

Again, based on the 2 sample $t$-test from the actual samples, the difference of the means is statistically significant. This means that we reject $H_0$ and assume that $H_A$ is true: <i>race affects the callback rate from employers.</i>

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

#### 4. Write a story describing the statistical significance in the context or the original problem.



From this statistical test, it was proven that race and the way a name is associated with race statistically significant in affecting the callback rate from the employers included in this dataset. 

#### 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?



This statistical analysis does not mean that race/name is the most important factor in callback success. No additional variables were analyzed, only the race/name and callback success rates were investigated. While it was determined that race/name affects the callback rate, other variables may have more influence on the success rate of callbacks from employers. For a more definitive answer to this question, all variables would need to be analyzed to determine their influence on the callback rate.